In [336]:
import numpy as np

# Rabin-Karp

In [347]:
x_RK = 2 # должно быть случайно от 2 до q!

def hashh(text, m=0, s='', q=(2**61-1), init=False):
    global x_RK
    
    if init:
        x_RK = np.random.randint(2, 10**10) # хочу randint(2, q), но не могу совладать с переполнением инта
        print('init by x_RK =', x_RK)
    elif not x_RK:
        raise BaseException('Initiate hashh function to start work!')
        
    if not m:
        m = len(text)
        
    init_arr = list(text[:m])
    pol_coef = np.power(x_RK, np.arange(m)[::-1])
    hs = np.sum(np.vectorize(ord)(init_arr) * pol_coef) % q
    text += '$'
    for i in range(len(text) - m):
        yield hs
        hs = ((hs - ord(text[i])*np.power(x_RK, m-1))*x_RK + ord(text[i+m])) % q

In [319]:
# x_RK = 0

# def hashh(text, m=0, s='', q=(2**61-1), init=False):
#     global x_RK
    
#     if init:
#         x_RK = np.random.randint(2, 10**10) # хочу randint(2, q), но не могу совладать с переполнением инта
#         print('init by x_RK =', x_RK)
#     elif not x_RK:
#         raise BaseException('Initiate hashh function to start work!')
        
#     if not m:
#         m = len(text)
        
#     init_arr = list(text[:m])
#     pol_coef = np.vectorize(lambda a: pow(x_RK, int(a), q))(np.arange(m)[::-1])
#     print(init_arr, pol_coef)
#     hs = np.sum(np.vectorize(ord)(init_arr) * pol_coef) % q
#     text += '$'
#     for i in range(len(text) - m):
#         yield hs
#         hs = ((hs - ord(text[i])*pow(x_RK, m-1, q))*x_RK + ord(text[i+m])) % q
#         print(text[i], text[i+m])

In [408]:
# можно еще искать сразу несколько одинаковой длины, отсортировав предварительно Patterns, доделаю

def RK_search(text, Patterns):
    finds = []
    for pat in Patterns:
        f = []
        hs_p = next(hashh(pat))
        i = 0
        for hs in hashh(text, len(pat)):
            if hs == hs_p:
                f.append(i)
            i += 1
        if len(f):
            finds.append(f)
        else:
            finds.append(['None'])
    return finds

In [410]:
text = 'aaabaababbaa'
Patterns = ['aab', 'ab', 'aaa', 'bab', 'bbb', 'abb']

finds = RK_search(text, Patterns)

print(text, '\n')
for i in range(len(Patterns)):
    print(Patterns[i], '\t:', *finds[i])

aaabaababbaa 

aab 	: 1 4
ab 	: 2 5 7
aaa 	: 0
bab 	: 6
bbb 	: None
abb 	: 7


# Knuth–Morris–Pratt

In [406]:
def pref(s):
    n = len(s)
    pr = np.zeros(n, dtype=np.int)
    for i in range(n-1):
        k = i
        while True:
            if s[i+1] == s[pr[k]]:
                pr[i+1] = pr[k] + 1
                break
            elif pr[k] == 0:
                pr[i+1] = 0
                break
            else:
                k = pr[k]
    return pr

In [440]:
def KMP_search(text, pattern):
    m = len(pattern)
    pref_patt = pref(pattern)
    i = 0
    k = 0
    while i in range(len(text)) and k in range(m):
        if text[i] == pattern[k]:
            i += 1
            k += 1
        elif k:
            k = pref_patt[k]
        else:
            i += 1
    if k == m:
        return i - m
    return -1

In [447]:
text = 'shsjkywabcdabscabcdabhskchabcdabscafhjsganabcdabscabcdabiakwjd'
pattern = 'abcdabscabcdabia'

i = KMP_search(text, pattern)
print('i =', i, '\nsubstring =', text[i:i+len(pattern)])

i = 42 
substring = abcdabscabcdabia


# Aho–Corasick